In [49]:
import pandas as pd
from tokenizers import decoders, models, normalizers, pre_tokenizers, processors, trainers, Tokenizer
from transformers import BertTokenizerFast
import numpy as np

In [40]:
df = pd.read_csv("/Users/pumpkin/Documents/Graduate_School/1st_Semester/ANLY_580/GU-ANLY580-PROJECT/cleaned_data/droppedDF.csv")
df = df.drop(columns=['Unnamed: 0'])

In [41]:
df['selftext'] = df['selftext'].astype('str') 

In [42]:
datasetLab = df

tokenizer = Tokenizer(models.WordPiece(unl_token="[UNK]"))

tokenizer.normalizer = normalizers.BertNormalizer(lowercase=True)

tokenizer.normalizer = normalizers.Sequence(
    [normalizers.NFD(), normalizers.Lowercase(), normalizers.StripAccents()]
)

tokenizer.pre_tokenizer = pre_tokenizers.BertPreTokenizer()

tokenizer.pre_tokenizer.pre_tokenize_str("This is an example!")

special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.WordPieceTrainer(vocab_size=25000, special_tokens=special_tokens)

batch_size = 1000

def batch_iterator():
    for i in range(0, len(datasetLab), batch_size):
        yield datasetLab[i : i + batch_size]["selftext"]

tokenizer.train_from_iterator(batch_iterator(), trainer=trainer)

cls_token_id = tokenizer.token_to_id("[CLS]")
sep_token_id = tokenizer.token_to_id("[SEP]")
print(cls_token_id, sep_token_id)

tokenizer.post_processor = processors.TemplateProcessing(
    single=f"[CLS]:0 $A:0 [SEP]:0",
    pair=f"[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
    special_tokens=[
        ("[CLS]", cls_token_id),
        ("[SEP]", sep_token_id),
    ],
)

encoding = tokenizer.encode("This is one sentence.", "With this one we have a pair.")

tokenizer.decoder = decoders.WordPiece(prefix="##")

new_tokenizer = BertTokenizerFast(tokenizer_object=tokenizer)

2 3


In [43]:
o = new_tokenizer(datasetLab['selftext'].tolist())['input_ids']
bert_token = pd.DataFrame(o)
datasetLab['domain'].replace({"explainlikeimfive": 0, "Showerthoughts": 1, "worldnews": 2,
                              "funny": 3, "pics": 4, "woahdude": 5,
                              "food": 6, "Jokes": 7, "AskReddit": 8,
                              "LifeProTips": 9, "books": 10, "todayilearned": 11,
                              "GetMotivated": 12, "movies": 13, "IAmA": 14}, inplace=True)

In [53]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [54]:
X = bert_token
Y = np.nan_to_num(datasetLab['domain'])

In [60]:
# split data into train and test sets
seed = 7
test_size = 0.10
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)
# fit model no training data
model = XGBClassifier(n_estimators = 100, max_depth = 6, min_child_weight = 1, eta = 0.3)
model.fit(X_train, y_train)
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

/Users/pumpkin/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:32:25] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy: 45.87%


In [61]:
from sklearn.decomposition import PCA

In [62]:
pca = PCA(n_components=2)

In [63]:
pca.fit(X)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [66]:
X

,0,1,2,3,4,5,6,7,8,9,...,9689,9690,9691,9692,9693,9694,9695,9696,9697,9698
0,2,48,1252.0,491.0,2051.0,413.0,382.0,1226.0,465.0,48.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,435,730.0,5386.0,2542.0,585.0,368.0,530.0,6974.0,414.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2274,413.0,40.0,906.0,17.0,3913.0,7557.0,388.0,14886.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,48,579.0,434.0,1076.0,388.0,5667.0,368.0,2579.0,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2,565,559.0,469.0,3067.0,377.0,393.0,6238.0,5136.0,4894.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4353,2,48,471.0,23751.0,409.0,18.0,48.0,454.0,410.0,10105.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4354,2,583,471.0,48.0,33.0,48.0,471.0,1101.0,18.0,5754.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4355,2,17,491.0,413.0,461.0,3071.0,33.0,17.0,574.0,1118.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4356,2,48,3413.0,423.0,3786.0,25.0,794.0,1275.0,6463.0,1722.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
X.to_csv("v.csv", index = False)